<a href="https://colab.research.google.com/github/KSY1526/myblog/blob/master/_notebooks/2021-12-23-Do_natural_language4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "[Do it 자연어] 4. 문장 쌍 분류하기 + 웹 실습"
- author: Seong Yeon Kim 
- categories: [book, jupyter, Do it, natural language, BERT, tokenizer, web, Classifier]

# 모델 환경설정

In [9]:
!pip install ratsnlp

In [10]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [11]:
from torch.cuda import is_available
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_task_name = 'pair-classification', # 문장 쌍 분류를 할 예정이므로
    downstream_corpus_name='klue-nli', # 업스테이지 기업이 공게한 KLUE-NLI 데이터로 파인튜닝
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls',
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs = 5,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed = 7,
)

kcbert-base 모델을 klue-nli 데이터로 파인튜닝 합니다.

In [12]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


set seed: 7


랜덤 시드와 로거를 설정합니다.

# 말뭉치 내려받기

In [13]:
nlpbook.download_downstream_dataset(args)

Downloading: 100%|██████████| 12.3M/12.3M [00:00<00:00, 37.5MB/s]
Downloading: 100%|██████████| 1.47M/1.47M [00:00<00:00, 35.4MB/s]


말뭉치를 내려받습니다.

In [14]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

토크나이저를 구축합니다.

# 학습 데이터 구축

In [15]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset

corpus = KlueNLICorpus() # json 파일형식의 KLUE-NLI 데이터를 문장(전제+가설)과 레이블(참 거짓 중립)으로 읽음 
train_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'train',
)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/klue-nli
INFO:ratsnlp:loading train data... LOOKING AT /root/Korpora/klue-nli/klue_nli_train.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 17.969 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 100분간 잘껄 그래도 소닉붐땜에 2점준다 + 100분간 잤다.
INFO:ratsnlp:tokens: [CLS] 100 ##분간 잘 ##껄 그래도 소 ##닉 ##붐 ##땜에 2 ##점 ##준다 [SEP] 100 ##분간 잤 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 8327, 15760, 2483, 4260, 8446, 1895, 5623, 5969, 10319, 21, 4213, 10172, 3, 8327, 15760, 2491, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], label=1)

KlueNLICorpus에서 받은 문장들을 미리 설정한 토크나이저로 분리합니다.

출력물은 input_ids, attention_mask, token_type_ids, label 총 4개가 나옵니다.

이전과 같은 결과인데 짧게 설명하며 input_ids은 토큰 시퀀스를, attention_mask는 패딩 여부를 알려줍니다.

token_type_ids은 세그먼트 정보로 첫번째 문장은 0, 두번째 문장은 1, 나머지 패딩은 0을 줍니다.

label은 0일때 참, 1일때 거짓, 2일때 중립을 의미합니다.

In [17]:
from torch.utils.data import DataLoader, RandomSampler

train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement = False),
    collate_fn = nlpbook.data_collator, # 뽑은 인스턴스를 배치로 바꿔줌(텐서 형태로)
    drop_last = False, 
    num_workers = args.cpu_workers,
)

학습 데이터 셋으로 로더를 구축했습니다. 배치크기만큼 인스턴스를 랜덤하게 뽑은 뒤 이를 합처 배치를 만듭니다. 

# 평가용 데이터 구축

In [18]:
from torch.utils.data import SequentialSampler

val_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'test',
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset), # 순서대로 추출
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /root/Korpora/klue-nli
INFO:ratsnlp:loading test data... LOOKING AT /root/Korpora/klue-nli/klue_nli_dev.json
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 1.457 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence A, B: 10명이 함께 사용하기 불편함없이 만족했다. + 10명이 함께 사용하기 불편함이 많았다.
INFO:ratsnlp:tokens: [CLS] 10명 ##이 함께 사용 ##하기 불편 ##함 ##없이 만족 ##했다 . [SEP] 10명 ##이 함께 사용 ##하기 불편 ##함이 많았 ##다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
INFO:ratsnlp:label: contradiction
INFO:ratsnlp:features: ClassificationFeatures(input_ids=[2, 21000, 4017, 9158, 9021, 8268, 10588, 4421, 8281, 14184, 8258, 17, 3, 21000, 4017, 9158, 9021, 8268, 10588, 11467, 14338, 4020, 17, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

입력받은 데이터를 토크나이저로 분리한 후 로더를 이용해 테스트용 배치를 만듭니다.

# 모델 학습

In [19]:
from transformers import BertConfig, BertForSequenceClassification

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = corpus.num_labels,
)

model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config = pretrained_model_config
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

pretrained_model_config은 프리트레인을 마친 BERT모델을 기록한 형태입니다.분류할 라벨이 3인것까지 정보를 줍니다.

model은 윗 모델에 문서 분류용 태스크 모듈을 덧붙인 모델입니다.

In [20]:
from ratsnlp.nlpbook.classification import ClassificationTask

task = ClassificationTask(model, args)

앞서 정의한 모델을 학습시킵니다. ClassificationTask 내에는 옵티마이저와 러닝 레이트 스케줄러가 있습니다.

In [21]:
trainer = nlpbook.get_trainer(args)
trainer

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-paircls exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


트레이너를 정의했습니다. 트레이너는 GPU/TPU 설정, 로그 및 체크포인트 등 귀찮은 설정을 알아서 해줍니다.

In [22]:
trainer.fit(
    task,
    train_dataloader = train_dataloader,
    val_dataloaders = val_dataloader
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

# 전제와 가설을 검증하는 웹 서비스 만들기

In [34]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(
    pretrained_model_name='beomi/kcbert-base',
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-paircls',
    max_seq_length=64,
)


downstream_model_checkpoint_fpath: /gdrive/My Drive/nlpbook/checkpoint-paircls/epoch=1-val_loss=0.82-v1.ckpt


인퍼런스 설정을 해줍니다.

In [35]:
import torch
from transformers import BertConfig, BertForSequenceClassification
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu'),
)

체크 포인트를 로드해줍니다. (이전에 만든 모델 업로드)

In [36]:
from transformers import BertConfig

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = 3
)

model = BertForSequenceClassification(pretrained_model_config)

BERT 설절을 로드하고 BERT 모델을 초기화합니다.

In [37]:
model.load_state_dict({k.replace('model.', ''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

초기화한 BERT 모델에 체크 포인트를 주입합니다. 그 후 모델을 평가모드로 바꿉니다.

In [38]:
def inference_fn(premise, hypothesis):
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length = args.max_seq_length,
        padding = 'max_length',
        truncation = True # 잘린 값 처리 여부
    )

    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        entailment_prob = round(prob[0][0].item(), 2)
        contradiction_prob = round(prob[0][1].item(), 2)
        neutral_prob = round(prob[0][1].item(), 2)

        if torch.argmax(prob) == 0:
            pred = '참'
        elif torch.argmax(prob) == 1:
            pred = '거짓'
        else:
            pred = '중립'

    return {
        'premise' : premise,
        'hypothesis' : hypothesis,
        'prediction' : pred,
        'entailment_data': f"참 {entailment_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'entailment_width': f"{entailment_prob*100}%",
        'contradiction_width': f"{contradiction_prob*100}%",
        'neutral_width': f"{neutral_prob*100}%",
    }

전제와 가설을 입력받아 각각 토큰화, 인덱싱을 수행한 것을 파이토치 텐서 자료형으로 변환한뒤 모델에 입력하는 함수를 만듭니다.

In [39]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8cea-35-225-219-137.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [23/Dec/2021 12:52:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 12:52:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Dec/2021 12:52:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 12:52:28] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Dec/2021 12:54:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 12:54:25] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Dec/2021 12:54:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 12:54:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Dec/2021 12:55:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 12:55:19] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Dec/2021 12:57:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Dec/2021 12:57:54] "GET /favicon.ico HTTP/1.1" 404 -


패키지를 설치하여 주어진 모델을 웹에서 서비스하게 해줍니다.

다만 오류가 뜨네요.. 이부분은 공부를 더 해야겠습니다.

# 느낀점

앞서 영화 감상평을 긍정, 부정으로 구분하는 모델, 이번에 전제와 가설이 일치하는지 여부를 판단하는 모델을 만들었습니다.

자연어의 기초 이론을 배우고 적용해봤는데, 언어를 수리적인 인풋으로 바꿔서 이를 판단하는 모델을 만든다는 것 자체가 신기했습니다.

다만, 제가 처음 자연어 처리를 떠올렸을때와 약간 다른 점은 사람이 개입할 여지가 조금 작다는 느낌이 들었습니다.

그래도 현실에서 유용한 모델을 만들 수 있다는 점이 신기했고, 더욱 더 공부하고 싶은 생각이 듭니다.